# Create Azure Sql Virtual Machine using Az-Cli

## PS Version and Modules
This notebook uses a <a href="https://youtu.be/BhbiAINQBYE">PowerShell Notebook kernel</a> (YouTube video) and requires PS v5.1 or greater. It also uses the latest version of Azure CLI. To install, see https://aka.ms/installazurecliwindows. Update instructions can be found at https://aka.ms/doc/UpdateAzureCliMsi

In [ ]:
# Check that PowerShell >= v5.1 and Azure Command Line Interface (Az CLI) are installed
if ($PSVersionTable.PSVersion.Major -lt 5)
{
    Write-Warning "Please install latest version of PowerShell. Do not continue executing the rest of the notebook."
}
else {
    $psVersion = $PSVersionTable.PSVersion.ToString()
    Write-Output "Found PS version... $psVersion"
    # Check for Azure CLI
    $azCli = (&az --version)    
    if ($azCli) {        
        $azVersion = $azCli[0].Split()[-1]
        Write-Output "Found Az CLI version... $azVersion"
    }
}

## Set Variables for the Notebook

In [ ]:
# ADP Resource 
$Env:BOOTSTRAP_Subscription     = ""                     # Azure Subscription ID/Name # The bacpac files and ADP Resources are assumed to be in the same subscription
$Env:BOOTSTRAP_ResourceGroup    = "TestResourceGroup"                     # Azure Resource Group which contains the ADP Resources

# SQL Server 
$VMName                     = "TestVM"                             # Name of the Virtual Machine to be created
$ImageSku                   = "Enterprise"                             # Choose your Image sku (see appendix)
$Location                   = "East US"                             # see Appendix for a list of location settings
$VmUsername                 = ""                                    # Username of Virtual Machine to be created
$VmPassword                 = ""                                    # Password of Virtual Machine to be created

# Set Variables for ADP Resources
$Env:BOOTSTRAP_FUNC            = $Env:BOOTSTRAP_ResourceGroup + "Control"   
$Env:BOOTSTRAP_BATCH           = $Env:BOOTSTRAP_ResourceGroup.ToLower() + "batch"
$Env:BOOTSTRAP_VNET            = $Env:BOOTSTRAP_ResourceGroup + "Vnet"

## Notebook Functions
Defines logical functions for the rest of the notebook. Function blocks are combined in a single cell that can be collapsed for readability or expanded for further examination. Nothing is executed until called later in the notebook. As a result, this cell is a requirement for any of the other cells below it. 

In [ ]:
# Expand cell to view framework

function Login-Azure
{    
    # query azure locations to test for existing az login session exists with valid access tocken
    $azureLocations = az account list-locations -o JSON 2>$null | ConvertFrom-Json
    
    if (!$azureLocations){ #If there are no az locations, there is no existing az login session
        $subscriptions = az login -o JSON | ConvertFrom-Json      # Login   
    }
    else {
        $subscriptions = az account list -o JSON | ConvertFrom-Json # getting subscriptions for the user to use in gridview
    }

    if(![string]::IsNullOrWhiteSpace($Env:BOOTSTRAP_Subscription)) #If there is a subscription specified by user in the variables section
    {
        $specified_Subscription= az account show --subscription $Env:BOOTSTRAP_Subscription -o json |ConvertFrom-Json 
        if (!$specified_Subscription) #if specified subscription is not valid
        {       
            $currentUser= az ad signed-in-user show --query "{displayName:displayName,UPN:userPrincipalName}" -o json|ConvertFrom-Json # get current logged in user infomration
            Write-Host "Refer below for the list of subscriptions for logged in account '$($currentUser.UPN)'`n"
            az account list --query "[].{Name:name,SubscriptionID:id}" -o table          # list subscriptions under current logged in account
        }
        else { # if specified subscription is valid
            Write-Output "Using subscription... '$($specified_Subscription.name)' ... '$($specified_Subscription.id)'" 
        }
    }
    else { # if no subscription is specified, users are given a gridview to select subscription from

        $selectedSubscription = $subscriptions | Select-Object -Property Name, Id | Out-GridView -PassThru
        $SubscriptionId = $selectedSubscription.Id
        $Subscription = $selectedSubscription.Name   
        $Env:BOOTSTRAP_Subscription = $subscription  
        Write-Output "Using subscription... '$Env:BOOTSTRAP_Subscription' ... '$SubscriptionId'"    
    }    
}

function Verify-ADPResources
{    
    [CmdletBinding()]
    param(
        [Parameter (Mandatory=$true)] [ValidateNotNullOrEmpty()] [string]$Subscription,
        [Parameter (Mandatory=$true)] [ValidateNotNullOrEmpty()] [string]$ADPResourceGroupName,
        [Parameter (Mandatory=$true)] [ValidateNotNullOrEmpty()] [string]$FunctionName,  
        [Parameter (Mandatory=$true)] [ValidateNotNullOrEmpty()] [string]$BatchAccountName,
        [Parameter (Mandatory=$true)] [ValidateNotNullOrEmpty()] [string]$VNetName,
        [Parameter (Mandatory=$false)] [ValidateNotNullOrEmpty()] [string]$ApplicationName="SqlPackageWrapper",  
        [Parameter (Mandatory=$false)] [ValidateNotNullOrEmpty()] [string]$ApplicationPackageVersionName="1",
        [Parameter (Mandatory=$false)] [ValidateNotNullOrEmpty()] [string]$SubNetName="default" 
    )    

# validate Subscription
$specified_Subscription= az account show --subscription $Subscription -o json | ConvertFrom-Json
if(!$specified_Subscription){
    $currentUser= az ad signed-in-user show --query "{displayName:displayName,UPN:userPrincipalName}" -o json|ConvertFrom-Json # get current logged in user information
    Write-Host "Refer below for the list of subscriptions for logged in account '$($currentUser.UPN)'`n"
    az account list --query "[].{Name:name,SubscriptionID:id}" -o table          # list subscriptions under current logged in account
    return } 
# validate ResourceGroup    
$specified_ResourceGroup= az group show -n $ADPResourceGroupName --subscription $Subscription -o json | ConvertFrom-Json
if(!$specified_ResourceGroup) {        
        return
        } 

$Installed = [ordered]@{} # ordered hash to store status of installation
$countError=0

#Verify if VNet exists       
$specified_VNet= az network vnet show -n $VNetName -g $ADPResourceGroupName --subscription $Subscription -o JSON 2>$null  |ConvertFrom-Json   
if(!$specified_VNet) {
    $Installed['VNET']="Not Found"
    $countError++
}   
else { 
    $existingVnetSubnet =  az network vnet subnet show -n $SubNetName --vnet-name $VNetName -g $ADPResourceGroupName --subscription $Subscription -o JSON 2>$null |ConvertFrom-Json
    if(!$existingVnetSubnet){
        $Installed['VNET']="Default Subnet under"+ $VNetName + "Not Found"
        $countError++
        }
    else {
    $Installed['VNET']="Installed"
    }
 }

#Verify if FunctionApp Exists
$specified_FunctionApp = az functionapp show -n $FunctionName -g $ADPResourceGroupName --subscription $Subscription -o JSON 2>$null | ConvertFrom-Json
if(!$specified_FunctionApp)
{
    $Installed['FunctionApp']="Not Installed"
    $countError++
}
else
{
    $Installed['FunctionApp']="Installed"
} 

#check if Batch account exists
$specified_BatchAccount = az batch account show -n $BatchAccountName -g $ADPResourceGroupName --subscription $Subscription -o JSON 2>$null | ConvertFrom-Json
if(!$specified_BatchAccount)
{
    $Installed['Batch']="Not Installed"
    $countError++
}
else
{
    $appPackageInstalled = az batch application package show --application-name $ApplicationName --version-name $ApplicationPackageVersionName -n $BatchAccountName -g $ADPResourceGroupName --subscription $Subscription -o JSON 2>$null | ConvertFrom-Json
    $connectedToStorage= $specified_BatchAccount.autoStorage 
    if($connectedToStorage -and $appPackageInstalled){ # BatchAccount connected to storageaccount and  applicationpackage is installed
            $Installed['Batch']="Installed"
            $Installed['Batch_ApplicationPackage']="Installed"
            $Installed['Batch_StorageAccount']="Connected to storage- "+$connectedToStorage.storageAccountId.Split("/")[-1]
        }
    if(!$connectedToStorage)
    {
        $Installed['Batch_StorageAccount']='Not Found'
        $countError++
    }  
    if(!$appPackageInstalled)
    {
        $Installed['Batch_ApplicationPackage']="Not Found"
        $countError++
    }    
}
if ($countError -gt 0){
    Write-Output "ADP Resources are not installed correctly. Please refer the list below and use the Bootstrap NB to install ADP Resources"
}
$Installed
if ($countError -eq 0){
    Write-Output "`nFound all ADP Resources."
}
}

function Prepare-InputForImportFunction
{    
    [CmdletBinding()]
    param(
        [Parameter (Mandatory=$true)] [ValidateNotNullOrEmpty()] [string]$Subscription,
        [Parameter (Mandatory=$true)] [ValidateNotNullOrEmpty()] [string]$ADPResourceGroupName,
        [Parameter (Mandatory=$true)] [ValidateNotNullOrEmpty()] [string]$FunctionName,  
        [Parameter (Mandatory=$true)] [ValidateNotNullOrEmpty()] [string]$BatchAccountName,
        [Parameter (Mandatory=$true)] [ValidateNotNullOrEmpty()] [string]$BackupFiles_StorageAccount,
        [Parameter (Mandatory=$true)] [ValidateNotNullOrEmpty()] [string]$BackupFiles_ContainerName,
        [Parameter (Mandatory=$true)] [ValidateNotNullOrEmpty()] [string]$VNetName,        
        [Parameter (Mandatory=$true)] [ValidateNotNullOrEmpty()] [string]$TargetRGName,
        [Parameter (Mandatory=$true)] [ValidateNotNullOrEmpty()] [string]$SqlServerName,
        [Parameter (Mandatory=$true)] [ValidateNotNullOrEmpty()] [string]$SqlServerPassword
    )
    
    $Result = @{}
    # Build Header 
    ## get Function key
    $FunctionAppID =az functionapp show -n $FunctionName -g $ADPResourceGroupName --subscription $Subscription --query "[id]" -o JSON 2>$null | ConvertFrom-Json
    $DefaultHostKey = az rest --method post --uri "$FunctionAppID/host/default/listKeys?api-version=2018-11-01" --query "[functionKeys.default]" -o JSON 2>$null | ConvertFrom-Json
    ## Build Json Object for Headers
    $headers = @{
        'x-functions-key' = $DefaultHostKey
    }
    $Result['Header']=$headers

    # Build string for Function URL 
    $specified_Subscription= az account show --subscription $Subscription -o json |ConvertFrom-Json #Get SpecifiedSubscriptionID
    $SubscriptionID= $specified_Subscription.id
    $FunctionUrl = 'https://'+ $FunctionName +'.azurewebsites.net/api/subscriptions/'+ $SubscriptionID +'/resourceGroups/' + $ADPResourceGroupName + '/Import'
    $Result['FunctionURL']=$FunctionUrl

    # Set parameter variables for Body
    ## Get BatchAccountURL 
    $specified_Batch = az batch account show -n $BatchAccountName -g $ADPResourceGroupName  --subscription $Subscription -o JSON 2>$null | ConvertFrom-Json
    $BatchAccountURL = 'https://' + $specified_Batch.accountEndpoint

    ## Get default SubNet ID for specified VNet
    $specified_VNet_SubNet = az network vnet subnet show -g $ADPResourceGroupName --vnet-name $VNetName -n 'default' --subscription $Subscription -o JSON |ConvertFrom-Json
    $VNetSubNetID = $specified_VNet_SubNet.id

    ## Create access token to source sql server
    $targetAccessToken = az account get-access-token --resource=https://database.windows.net --query accessToken
    $targetAccessToken

    ## Build JSon object for Body
    $Body = @{
        batchAccountUrl = $BatchAccountURL
        VNetSubnetId= $VNetSubNetID
        storageAccountName = $BackupFiles_StorageAccount
        containerName = $BackupFiles_ContainerName
        targetSqlServerResourceGroupName = $TargetRGName
        targetSqlServerName = $SQLServerName 
        userName = $SqlServerLogin 
        targetAccessToken = $targetAccessToken
        sqlAdminPassword = $SqlServerPassword
    }
    $json = $Body | ConvertTo-Json
    $Result['Body']=$json

    $Result
    
}

function Provision-FuncRBAC {
    [CmdletBinding()]
    param (
        [Parameter(Mandatory=$true)][ValidateNotNullOrEmpty()][string]$Subscription,
        [Parameter(Mandatory=$true)][ValidateNotNullOrEmpty()][string]$ResourceGroupName,
        [Parameter(Mandatory=$true)][ValidateNotNullOrEmpty()][string]$FunctionName,
        [Parameter(Mandatory=$true)][ValidateNotNullOrEmpty()][string]$ScopeRGName,
        [Parameter(Mandatory=$false)][ValidateNotNullOrEmpty()][string]$Role="Contributor"
    )

    # Get the scope resource group's ID
    $scopeID = az group show --resource-group $ScopeRGName --subscription $Subscription --query "[id]" -o JSON | ConvertFrom-Json 
    if(!$scopeID) {
        Write-Output "Provision-FuncRBAC failed." 
        return }
    else { Write-Output "Found scope '$ScopeRGName' with ID... '$scopeID'"
    }

    # Get the az function principal id
    $app_PrincipalID = az functionapp show -n $FunctionName --resource-group $ResourceGroupName --subscription $Subscription --query "[identity.principalId]" -o JSON  | ConvertFrom-Json  
    if(!$app_PrincipalID) {
        Write-Output "Provision-FuncRBAC failed." 
        return }
    else {  Write-Output "Found principal id of Azure function '$FunctionName'... '$app_PrincipalID'"
    }

    # Verify if a role assignment has been created for function
    $app_RoleAssignmentDefinition= az role assignment list --subscription $Subscription --assignee $app_PrincipalID --scope $scopeID --query "[].roleDefinitionName" -o JSON 2>$null  | ConvertFrom-Json

    if($app_RoleAssignmentDefinition -eq $Role)
    {
        Write-Output "Found Role Assignment for Principal ID.. '$app_PrincipalID' with Role.. '$app_RoleAssignmentDefinition' . No work needed"
    }
    else
    {
        # Continue to setup RBAC, once we verify an assignment is not setup and all the resources exist
        Write-Output "Creating new role assignment by running: 'az functionapp identity assign -n $FunctionName --role $Role -g $ResourceGroupName --scope $scopeID --subscription $Subscription'"
        Write-Warning "If your account does not have the access to assign new roles as Owner or User Access Administrator for the resource group, than you will need to contact your Azure AD Administrator to assign a service principle using the commands above"
        az functionapp identity assign -n $FunctionName --role $Role -g $ResourceGroupName --scope $scopeID --subscription $Subscription 
    }
}
Write-Host "Helper Functions Created successfully"  

## Connect to Azure Account
Run the below cell to login to an Azure account. <b>Be sure to check the Windows Taskbar for a login dialog box underneath the notebook or other windows or by pressing Alt+TAB.</b>

In [ ]:
Login-Azure

## Verify ADP Resources 
Verify if ADP resources exists in specified Resource Group

In [ ]:
Verify-ADPResources -Subscription $Env:BOOTSTRAP_Subscription -ADPResourceGroupName $Env:BOOTSTRAP_ResourceGroup `
                     -BatchAccountName $Env:BOOTSTRAP_BATCH -FunctionName $Env:BOOTSTRAP_FUNC -VNetName $Env:BOOTSTRAP_VNET 

## Create a resource group

In Azure, all resources are allocated in a resource management group. Resource groups provide logical groupings of resources that make them easier to work with as a collection

In [ ]:
$rsgExists = az group exists -n $Env:BOOTSTRAP_ResourceGroup
if ($rsgExists -eq 'false') {
    az group create -l $Location -n $Env:BOOTSTRAP_ResourceGroup
}

## Create a virtual machine



In [ ]:
az vm create -n sql -g $Env:BOOTSTRAP_ResourceGroup --image MicrosoftSQLServer:SQL2017-WS2016:Standard:14.0.1000200 --location $Location --admin-username $VmUsername --admin-password $VmPassword --verbose

## Get VM information with queries
Now that a VM has been created, detailed information about it can be retrieved. The common command for getting information from a resource is _show_

In [ ]:
az vm show --name TutorialVM1 --resource-group TutorialResources

Lot of information can be seen, which can be difficult to parse visually. The returned JSON contains information on authentication, network interfaces, storage, and more. Most importantly, it contains the Azure object IDs for resources that the VM is connected to. Object IDs allow accessing these resources directly to get more information about the VM's configuration and capabilities.

In order to extract the object ID , the --query argument is used. Queries are written in the JMESPath query language. Start with getting the network interface controller (NIC) object ID

In [ ]:
az vm show --name TutorialVM1 \
  --resource-group TutorialResources \
  --query 'networkProfile.networkInterfaces[].id' \
  --output tsv

### Appendix: Locations
See the <a href="https://azure.microsoft.com/en-us/global-infrastructure/locations/">Azure locations</a> page for a complete list of Azure regions along with their general physical location. The following is a list of common North American location settings for this guide:

#### US Regions
| Setting | Location |
| ------------ | --------- |
| Central US | Iowa |
| East US | Virginia |
| East US 2 | Virginia |
| North Central US | Illinois |
| South Central US | Texas |
| West US 2 | Washington |
| West Central US | Wyoming |
| West US | California | 
| Canada Central | Toronto |
| Canada East | Quebec City |
| Brazil South | Sao Paulo |
| Mexico Central | Queretaro |

### Appendix: Storage Skus
Use these as values for provisioning storage skus. 

Data for table taken from <a href="https://docs.microsoft.com/en-us/rest/api/storagerp/srp_sku_types">SKU Types</a> page but is subject to change. Not all skus are listed here. SKU type names are case-sensitive.

| Name |
| -----|
| Developer |
| Enterprise |
| Express |
| Standard |
| Web |